In [1]:
import pandas as pd
import pickle
import sys
import os
import time
import numpy as np
sys.path.append("/home/ziniu.wzn/BayesCard")
from DataPrepare.schemas import gen_DB0_schema
schema = gen_DB0_schema("/home/ziniu.wzn/AttEst_support/")
ensemble_location = "/home/ziniu.wzn/AttEst_support/DB0/BN_ensemble/"
from DataPrepare.query_prepare_BayesCard import prepare_join_queries

In [2]:
query_filename = "/home/ziniu.wzn/AttEst_support/DB0/join_queries.sql"
parsed_queries, true = prepare_join_queries(schema, ensemble_location, pairwise_rdc_path=None, 
                                            query_filename=query_filename, true_card_exist=True)

In [3]:
from Models.BN_ensemble_model import BN_ensemble
def load_ensemble(schema, model_path=ensemble_location):
    bn_ensemble = BN_ensemble(schema)
    for file in os.listdir(model_path):
        if file.endswith(".pkl"):
            print(file)
            with open(model_path+file, "rb") as f:
                bn = pickle.load(f)
                bn.infer_algo = "exact-jit"
                bn.init_inference_method()
            bn_ensemble.bns.append(bn)
            #bn_ensemble.bns[int(file[0])] = bn
    return bn_ensemble
bn_ensemble = load_ensemble(schema)

2_chow-liu_1.pkl
4_chow-liu_1.pkl
8_chow-liu_1.pkl
3_chow-liu_1.pkl
1_chow-liu_1.pkl
6_chow-liu_1.pkl
7_chow-liu_1.pkl
9_chow-liu_1.pkl
5_chow-liu_1.pkl
0_chow-liu_1.pkl


In [4]:
with open(query_filename, "rb") as f:
    real_query = f.readlines()
queries = bn_ensemble.parse_query_all(parsed_queries)

In [5]:
latency = []
q_errors = []
for i, q in enumerate(queries):
    tic = time.time()
    try:
        pred = bn_ensemble.cardinality(q)
    except:
        print(f"query {i} is invalid!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        # this query itself is invalid or it is not recognizable by the learnt BN
        continue
    latency.append(time.time() - tic)
    if pred is None or pred <= 1:
        pred = 1
    error = max(pred / true[i], true[i] / pred)
    print(f"predicting query no {i}: {real_query[i]} \n")
    print(f"true cardinality {true[i]}, predicted {pred} with q-error {error}")
    q_errors.append(error)
print("=====================================================================================")
for i in [50, 90, 95, 99, 100]:
    print(f"q-error {i}% percentile is {np.percentile(q_errors, i)}")
print(f"average latency is {np.mean(latency)*1000} ms")

predicting query no 0: b'990||SELECT COUNT(*) FROM table0 AS t0, table1 AS t1, table3 AS t3, table4 AS t4, table5 AS t5, table6 AS t6 WHERE t5.table0Id = t0.Id AND t3.table0Id = t0.Id AND t1.table0Id = t0.Id AND t6.table1Id2 = t1.Id AND t4.table1Id = t1.Id AND t0.attr1 >= 10632153 AND t0.attr1 <= 127429994 AND t0.attr6 <= 6948 AND t1.attr2 >= -19.0 AND t1.attr3 >= 12.0 AND t3.attr3 = 1.0 AND t4.attr0 = 19.0 AND t5.attr2 >= 108872.0 AND t5.attr2 <= 130998281.0 AND t6.attr0 >= 23098947;\n' 

true cardinality 990, predicted 58.956711131027426 with q-error 16.791981455678386
predicting query no 1: b'41||SELECT COUNT(*) FROM table0 AS t0, table1 AS t1, table3 AS t3, table4 AS t4, table5 AS t5, table6 AS t6 WHERE t6.table1Id2 = t1.Id AND t5.table1Id = t1.Id AND t3.table1Id = t1.Id AND t4.table1Id = t1.Id AND t3.table0Id = t0.Id AND t0.attr0 >= 19 AND t1.attr2 <= 192.0 AND t1.attr4 >= 0.0 AND t1.attr5 >= 0.0 AND t1.attr5 <= 45.0 AND t3.attr3 = 16.0 AND t4.attr4 = 6.0 AND t5.attr2 >= 1179241.0

predicting query no 17: b'69851||SELECT COUNT(*) FROM table0 AS t0, table1 AS t1, table3 AS t3, table4 AS t4 WHERE t3.table0Id = t0.Id AND t3.table1Id = t1.Id AND t4.table0Id = t0.Id AND t0.attr5 >= 2 AND t0.attr6 >= 4 AND t0.attr6 <= 6948 AND t1.attr1 >= 4007557.0 AND t3.attr1 >= -45446400.0 AND t3.attr1 <= 129600000.0 AND t4.attr3 = 10.0;\n' 

true cardinality 69851, predicted 18639.2593151546 with q-error 3.747520157263322
predicting query no 18: b'3420||SELECT COUNT(*) FROM table0 AS t0, table1 AS t1, table2 AS t2, table3 AS t3, table4 AS t4, table5 AS t5, table6 AS t6 WHERE t2.table0Id = t0.Id AND t1.table0Id = t0.Id AND t4.table0Id = t0.Id AND t3.table0Id = t0.Id AND t6.table1Id2 = t1.Id AND t5.table1Id = t1.Id AND t0.attr3 >= 3 AND t0.attr4 <= 214 AND t1.attr1 >= 100814.0 AND t1.attr5 >= 1.0 AND t1.attr5 <= 45.0 AND t2.attr0 >= 5395110 AND t2.attr0 <= 128696985 AND t3.attr0 = 11.0 AND t3.attr1 >= -40003200.0 AND t3.attr1 <= 131068800.0 AND t4.attr1 >= 2564220.0 AND t4.attr1 <= 1

predicting query no 38: b'3072||SELECT COUNT(*) FROM table0 AS t0, table1 AS t1, table2 AS t2, table3 AS t3, table4 AS t4, table5 AS t5, table6 AS t6 WHERE t2.table0Id = t0.Id AND t5.table0Id = t0.Id AND t3.table0Id = t0.Id AND t1.table0Id = t0.Id AND t6.table1Id2 = t1.Id AND t4.table1Id = t1.Id AND t0.attr3 >= 6 AND t1.attr4 >= 1.0 AND t2.attr0 <= 130455655 AND t3.attr1 >= -45964800.0 AND t3.attr1 <= 130464000.0 AND t4.attr1 <= 130254945.0 AND t4.attr3 = 8.0 AND t4.attr4 = 3.0 AND t5.attr2 >= 118606.0 AND t6.attr0 >= 24658873 AND t6.attr0 <= 130786995;\n' 

true cardinality 3072, predicted 65789250.13844612 with q-error 21415.771529442096
predicting query no 39: b'34||SELECT COUNT(*) FROM table0 AS t0, table1 AS t1, table4 AS t4, table5 AS t5 WHERE t5.table0Id = t0.Id AND t5.table1Id = t1.Id AND t4.table0Id = t0.Id AND t0.attr0 >= 23 AND t0.attr4 >= 1 AND t0.attr4 <= 1920 AND t1.attr2 <= 184.0 AND t1.attr3 >= 44.0 AND t4.attr2 = 15.0 AND t5.attr0 = 8.0;\n' 

true cardinality 34, predi

In [6]:
import numpy as np
import time

In [15]:
t = time.time()
x = np.random.randint(40000, size = 10000000)
unique, counts = np.unique(x, return_counts=True)
time.time() - t

0.7185041904449463

In [14]:
counts[np.argsort(counts)]

array([2332, 2334, 2338, ..., 2656, 2656, 2679])

In [20]:
np.concatenate([np.ones(5)])


array([1., 1., 1., 1., 1.])

In [22]:
a = np.array([])

In [27]:
a = np.arange(5)

In [30]:
np.searchsorted(a, -1)

0

In [26]:
a

array([], dtype=float64)